In [18]:
# !pip install keras_ocr
import math
import matplotlib.pyplot as plt
import keras_ocr
import numpy as np
import cv2
pipeline = keras_ocr.pipeline.Pipeline()#read image from the an image path (a jpg/png file or an image url)
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)
def inpaint_text(img_path, pipeline):
    # read image
    img = keras_ocr.tools.read(img_path)
    # generate (word, box) tuples
    words=[]
    boxes=[]
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
           x0, y0 = box[1][0]
           x1, y1 = box[1][1]
           x2, y2 = box[1][2]
           x3, y3 = box[1][3]
        #    box[1].tolist()
        #    box[1].to(list)
           words.append(box[0])
           boxes.append(box[1].tolist())
           x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
           x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)

           thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))

           cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,
           thickness)
           img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
           img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
           cv2.imwrite(f'removed_text_data/{img_path}',img_rgb)

    return(img_rgb,words,boxes,prediction_groups,mask)

Looking for /home/kushal/.keras-ocr/craft_mlt_25k.h5
Looking for /home/kushal/.keras-ocr/crnn_kurapan.h5


In [19]:
import cv2
import json
# !cp -r drive/MyDrive/data1/img data/img

text_removed={}
i=int(0)
with open("data/train.jsonl", 'r') as f:
    for line in f:
        entry = json.loads(line)
        if (i==100):
            with open("text_removed.json", 'w') as f:
                  json.dump(text_removed, f)
            i=0
        else:
            i+=1
    # Iterate through each image entry in the JSON data
        # print(entry)
        # image_id = int(entry['id'])
        image_path = "data/"+entry['img']
        # label = int(entry['label'])
        # text = entry['text']
        # image_path = "drive"
        imgs,words,boxes,prediction_group,mask=inpaint_text(image_path,pipeline)
        text_removed[int(entry['id'])]={
            'id' : int(entry['id']),
            'words' : words,
            'boxes' : boxes,
            # 'prediction_group' : prediction_group,
            # 'mask' : mask,
        }


1/1 [==============================] - 8s 8s/step


KeyboardInterrupt: 